# Рекуррентные нейронные сети

## NeuroWorkshop

Дмитрий Сошников | dmitri@soshnikov.com

### Какие задачи мы умеем решать

До этого момента мы сталкивались с нейронными сетями, размерность входных данных для которых была фиксирована:
* Координаты точек
* Изображения

Но есть другие классы задач:
* Определение тональности текста
* Генерация текста в заданном стиле
* Описание содержимого фотографии

## Задача

Решим модельную задачу: определение тональности слова. Для этого скачаем словари позитивных и негативных слов, и попробуем предсказывать, является ли слово позитивным или негативным, побуквенно.

 > Внимание! Данная задача - учебная. На практике аналогичная задача с помощью нейросетей хорошо решается для определения тональности предложений, разбитых на слова.

In [1]:
if not os.path.exists('sentiment-train.txt'):
    !wget https://raw.githubusercontent.com/shwars/NeuroWorkshop/master/Data/Sentiment/sentiment-train.txt
    !wget https://raw.githubusercontent.com/shwars/NeuroWorkshop/master/Data/Sentiment/sentiment-test.txt

--2017-11-26 14:15:45--  https://raw.githubusercontent.com/shwars/NeuroWorkshop/master/Data/Sentiment/sentiment-train.txt
Resolving webproxy (webproxy)... 10.72.8.104
Connecting to webproxy (webproxy)|10.72.8.104|:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 73380 (72K) [text/plain]
Saving to: 'sentiment-train.txt.4'

sentiment-train.txt 100%[===================>]  71.66K  --.-KB/s    in 0.04s   

2017-11-26 14:15:45 (1.71 MB/s) - 'sentiment-train.txt.4' saved [73380/73380]

--2017-11-26 14:15:45--  https://raw.githubusercontent.com/shwars/NeuroWorkshop/master/Data/Sentiment/sentiment-test.txt
Resolving webproxy (webproxy)... 10.72.8.104
Connecting to webproxy (webproxy)|10.72.8.104|:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 8833 (8.6K) [text/plain]
Saving to: 'sentiment-test.txt.4'

sentiment-test.txt. 100%[===================>]   8.63K  --.-KB/s    in 0.002s  

2017-11-26 14:15:46 (3.78 MB/s) - 'sentiment-test.txt.4' save

In [2]:
from __future__ import print_function
import numpy as np
import os
import sys
from cntk import Trainer
import cntk as C
from cntk.learners import sgd, learning_rate_schedule, UnitType
from cntk.ops import sequence
from cntk.losses import cross_entropy_with_softmax
from cntk.metrics import classification_error
from cntk.layers import LSTM, Stabilizer, Recurrence, Dense, For, Sequential
from cntk.logging import log_number_of_parameters, ProgressPrinter

def read(fn):
    f = open(fn,encoding="utf-8").readlines()
    return list(map(lambda s: s.split(",")[0],f)),list(map(lambda s: int(s.split(",")[1].strip()),f))

In [5]:
def char_to_num(c):
    if (c>='a' and c<='z'): return ord(c)-ord('a')
    else: return 0;

def num_to_char(n):
    return chr(ord('a')+n)

def to_onehot(n):
    return np.eye(vocab_size,dtype=np.float32)[n]

In [6]:
words, labels = read("sentiment-train.txt")

print(words[0:5]); 
print(labels[0:5])

['plebeian', 'stupendously', 'cripples', 'ingrate', 'sharper']
[-1, 1, -1, -1, 1]


### Что делаем с переменной размерностью?

Чтобы подать слово на вход сети, нам нужно будет дополнить каждое слово до максимальной длины. А также параллельно преобразуем в 1-hot encoding.

In [7]:
input_size = max(map(len,words)); vocab_size = char_to_num('z')+1

def fill(l):
    if (len(l)<input_size): return [0]*(input_size-len(l))+l 
    else: return l

In [8]:
words_arr = np.array([(to_onehot(fill(list(map(char_to_num,list(w)))))) for w in words])
labels_arr = np.array([np.array([0,1],dtype=np.float32) if x==-1 else np.array([1,0],dtype=np.float32) for x in labels])
print(words_arr[0:2])
print(labels_arr[0:2])

[[[ 1.  0.  0. ...,  0.  0.  0.]
  [ 1.  0.  0. ...,  0.  0.  0.]
  [ 1.  0.  0. ...,  0.  0.  0.]
  ..., 
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 1.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]]

 [[ 1.  0.  0. ...,  0.  0.  0.]
  [ 1.  0.  0. ...,  0.  0.  0.]
  [ 1.  0.  0. ...,  0.  0.  0.]
  ..., 
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  1.  0.]]]
[[ 0.  1.]
 [ 1.  0.]]


Задаём конфигурацию сети. Входная переменная имеет размерность $\mathit{макс\ длина\ слова} \times \mathit{число\ букв}$, выходная - 2.

In [9]:
input_var = C.input_variable((input_size,vocab_size))
label_var = C.input_variable((2))

model = Sequential([Dense(500,activation=C.ops.relu),Dense(2,activation=None)])
z = model(input_var); z_sm = C.softmax(z)

ce = cross_entropy_with_softmax(z, label_var)
errs = classification_error(z, label_var)

In [10]:
lr_per_sample = learning_rate_schedule(0.02, UnitType.minibatch)
learner = C.learners.sgd(z.parameters, lr_per_sample)
progress_printer = ProgressPrinter(freq=100, tag='Training')
trainer = Trainer(z, (ce, errs), learner, progress_printer)

    
log_number_of_parameters(z)

minibatch_size=10

Training 313502 parameters in 4 parameter tensors.


Тренируем сеть на всех выборках, в течение нескольких эпох.

In [11]:
for ep in range(20):
    print("Epoch={}".format(ep))
    for mb in range(0,len(words),minibatch_size):
        trainer.train_minibatch({input_var: words_arr[mb:mb+minibatch_size], label_var: labels_arr[mb:mb+minibatch_size]})

Epoch=0
Learning rate per minibatch: 0.02
 Minibatch[   1- 100]: loss = 0.611355 * 1000, metric = 29.60% * 1000;
 Minibatch[ 101- 200]: loss = 0.581246 * 1000, metric = 27.40% * 1000;
 Minibatch[ 201- 300]: loss = 0.636209 * 1000, metric = 32.50% * 1000;
 Minibatch[ 301- 400]: loss = 0.588381 * 1000, metric = 27.90% * 1000;
 Minibatch[ 401- 500]: loss = 0.602272 * 1000, metric = 29.80% * 1000;
 Minibatch[ 501- 600]: loss = 0.610872 * 1000, metric = 29.90% * 1000;
Epoch=1
 Minibatch[ 601- 700]: loss = 0.599966 * 994, metric = 30.08% * 994;
 Minibatch[ 701- 800]: loss = 0.575552 * 1000, metric = 27.60% * 1000;
 Minibatch[ 801- 900]: loss = 0.616348 * 1000, metric = 32.60% * 1000;
 Minibatch[ 901-1000]: loss = 0.569062 * 1000, metric = 27.40% * 1000;
 Minibatch[1001-1100]: loss = 0.601695 * 1000, metric = 30.70% * 1000;
 Minibatch[1101-1200]: loss = 0.599509 * 1000, metric = 29.70% * 1000;
Epoch=2
 Minibatch[1201-1300]: loss = 0.584021 * 994, metric = 28.67% * 994;
 Minibatch[1301-1400]: 

 Minibatch[11301-11400]: loss = 0.472238 * 1000, metric = 21.50% * 1000;
 Minibatch[11401-11500]: loss = 0.455855 * 1000, metric = 21.20% * 1000;
Epoch=19
 Minibatch[11501-11600]: loss = 0.470856 * 994, metric = 21.43% * 994;
 Minibatch[11601-11700]: loss = 0.451102 * 1000, metric = 20.00% * 1000;
 Minibatch[11701-11800]: loss = 0.448910 * 1000, metric = 19.60% * 1000;
 Minibatch[11801-11900]: loss = 0.430465 * 1000, metric = 20.90% * 1000;
 Minibatch[11901-12000]: loss = 0.463258 * 1000, metric = 21.10% * 1000;
 Minibatch[12001-12100]: loss = 0.453039 * 1000, metric = 20.90% * 1000;


Теперь протестируем результаты на тестовой выборке

In [12]:
def check(net,dofill=True):
    total = 0
    correct = 0
    for w,l in zip(words_test,labels_test):
        if dofill: w_a = to_onehot(fill(list(map(char_to_num,list(w)))))
        else: w_a = to_onehot(list(map(char_to_num,list(w))))
        out = net(w_a)[0]
        if (out[1]>0.5 and l==-1): correct+=1
        if (out[0]>0.5 and l==1): correct+=1
        total+=1
    print("{} out of {} correct ({}%)".format(correct,total,correct/total*100))

In [13]:
words_test, labels_test = read("sentiment-test.txt")

check(z_sm)


470 out of 725 correct (64.82758620689654%)


## Выводы

 * Иногда можно решить задачу переменной размерности с помощью расширения до фиксированной размерности
 * Результат не всегда хороший
 * Некоторые задачи требуют переменной размерности

## Рекуррентные сети

> **Рекуррентная сеть** (RNN) - это сеть для анализа последовательностей с учетом *времени*, в которой выход сети в момент $t$ подаётся на вход сети в момент $t+1$ 

 * Выход сети, подающийся на вход, описывает *состояние* сети
 * Последовательность переменного размера представляется вектором состояния фиксированного размера

## Пример: классификация текста

![](https://raw.githubusercontent.com/shwars/NeuroWorkshop/master/images/RNN-1.JPG)

## Основная идея

<img src="https://raw.githubusercontent.com/shwars/NeuroWorkshop/master/images/RNN-2.JPG" width="70%"/>

Слова подаются по-очереди на ячейки сети, которые перерабатывают состояние, обобщающее свойство последовательности вплоть до текущего момента.

Когда все слова обработаны, обобщающий вектор подаётся на вход обычной полносвязной сети (многослойному персептрону).

<img src="https://raw.githubusercontent.com/shwars/NeuroWorkshop/master/images/RNN-3.JPG" width="70%"/>

В результате для последовательности данной длины сеть разворачивается в такую структуру. В каждом квадратике используются матрицы с общими весами (weight sharing).

<img src="https://raw.githubusercontent.com/shwars/NeuroWorkshop/master/images/RNN-unroll.JPG" width="70%"/>

## Рекуррентная сеть

<img src="https://raw.githubusercontent.com/shwars/NeuroWorkshop/master/images/RNN-recursive.JPG" width="30%" aligh="left"/>
$$
s_{t+1}=f(s_t^T\times W_{hh}+x_t^T\times W_{xh}+b)\\
y_t = MLP(s_t)
$$

## RNN в CNTK

Для описания RNN в CNTK предусмотрен специальный механизм: **динамические измерения** (dynamic axes).

  * Динамическое измерение - это измерение, по которому заранее неизвестна размерность
  * По умолчанию есть одно динамическое измерение - minibatch axis
  * Допускается ещё одно динамическое измерение - для последовательностей переменной длины

In [14]:
words_arr1 = [to_onehot(list(map(char_to_num,list(w)))) for w in words]

input_var = sequence.input_variable(vocab_size)
label_var = C.input_variable(2)

model = Sequential([Recurrence(C.layers.RNNStep(200,activation=C.relu)),sequence.last,Dense(100,activation=C.relu),Dense(2)])

z = model(input_var)
z_sm = C.softmax(z)

In [15]:
ce = cross_entropy_with_softmax(z, label_var)
errs = classification_error(z, label_var)

lr_per_sample = learning_rate_schedule(0.02, UnitType.minibatch)
learner = C.learners.sgd(z.parameters, lr_per_sample)
progress_printer = ProgressPrinter(freq=100, tag='Training')
trainer = Trainer(z, (ce, errs), learner, progress_printer)

log_number_of_parameters(z)

minibatch_size = 10

Training 65702 parameters in 7 parameter tensors.


In [16]:
for ep in range(20):
    print("Epoch={}".format(ep))
    for mb in range(0, len(words), minibatch_size):
        trainer.train_minibatch(
            {input_var: words_arr1[mb:mb + minibatch_size], label_var: labels_arr[mb:mb + minibatch_size]})

Epoch=0
Learning rate per minibatch: 0.02
 Minibatch[   1- 100]: loss = 0.626477 * 1000, metric = 32.00% * 1000;
 Minibatch[ 101- 200]: loss = 0.586221 * 1000, metric = 27.40% * 1000;
 Minibatch[ 201- 300]: loss = 0.629491 * 1000, metric = 32.20% * 1000;
 Minibatch[ 301- 400]: loss = 0.594477 * 1000, metric = 27.90% * 1000;
 Minibatch[ 401- 500]: loss = 0.610718 * 1000, metric = 29.90% * 1000;
 Minibatch[ 501- 600]: loss = 0.610683 * 1000, metric = 29.90% * 1000;
Epoch=1
 Minibatch[ 601- 700]: loss = 0.606643 * 994, metric = 30.08% * 994;
 Minibatch[ 701- 800]: loss = 0.581229 * 1000, metric = 27.60% * 1000;
 Minibatch[ 801- 900]: loss = 0.624258 * 1000, metric = 32.10% * 1000;
 Minibatch[ 901-1000]: loss = 0.583406 * 1000, metric = 27.40% * 1000;
 Minibatch[1001-1100]: loss = 0.615405 * 1000, metric = 31.00% * 1000;
 Minibatch[1101-1200]: loss = 0.606095 * 1000, metric = 29.90% * 1000;
Epoch=2
 Minibatch[1201-1300]: loss = 0.592496 * 994, metric = 28.87% * 994;
 Minibatch[1301-1400]: 

 Minibatch[11301-11400]: loss = 0.297102 * 1000, metric = 12.60% * 1000;
 Minibatch[11401-11500]: loss = 0.320914 * 1000, metric = 13.30% * 1000;
Epoch=19
 Minibatch[11501-11600]: loss = 0.312036 * 994, metric = 13.08% * 994;
 Minibatch[11601-11700]: loss = 0.294523 * 1000, metric = 12.30% * 1000;
 Minibatch[11701-11800]: loss = 0.280687 * 1000, metric = 11.90% * 1000;
 Minibatch[11801-11900]: loss = 0.290301 * 1000, metric = 11.70% * 1000;
 Minibatch[11901-12000]: loss = 0.282102 * 1000, metric = 13.10% * 1000;
 Minibatch[12001-12100]: loss = 0.268883 * 1000, metric = 11.20% * 1000;


## Проверяем результат

In [17]:
check(z_sm,False)

506 out of 725 correct (69.79310344827586%)


## Виды рекуррентных сетей

|<img src="https://raw.githubusercontent.com/shwars/NeuroWorkshop/master/images/RNN-classify.JPG" width="50%"/>| Классификация |
|------|-------|
|<img src="https://raw.githubusercontent.com/shwars/NeuroWorkshop/master/images/RNN-generate.JPG" width="50%"/> | Генерация |

## Виды рекуррентных сетей

|<img src="https://raw.githubusercontent.com/shwars/NeuroWorkshop/master/images/RNN-compare.JPG" width="50%"/>| Сравнение |
|------|-------|
|<img src="https://raw.githubusercontent.com/shwars/NeuroWorkshop/master/images/RNN-seq2seq.JPG" width="50%"/> | Перевод |

## Классификация сетей по A.Karpathy

<img src="https://raw.githubusercontent.com/shwars/NeuroWorkshop/master/images/RNN-Karpathy.JPG" width="80%"/>

## Проблемы RNN

 * Vanishing/Exloding Gradient Problem
 * Проблема "забывания"
 
<img src="https://raw.githubusercontent.com/shwars/NeuroWorkshop/master/images/RNN-problem.JPG" width="50%"/>

## LSTM: Long Short-Term Memory

Решение проблемы: специальная архитектура ячеек сети:

 * Передаём между ячейками отдельный вектор состояния
 * В явном виде задаём операции забывания какой-то части состояния и переноса информации из входных данных в состояние
 * Реализуется посредством вентилей (gates): input gate, forget gate, output gate

<table><tr><td>Обычная RNN</td><td>LSTM</td></tr>
<tr><td>
 <img src="https://raw.githubusercontent.com/shwars/NeuroWorkshop/master/images/colah/LSTM3-SimpleRNN.png"/>
</td><td>
 <img src="https://raw.githubusercontent.com/shwars/NeuroWorkshop/master/images/colah/LSTM3-chain.png"/>
</td></tr></table>

Жёлтый квадратик - полносвязная сеть с функцией активации.

## Работа LSTM-ячейки

 * Forget Gate - на основе входной информации решает, какую часть информации в векторе состояния забыть (левая $\sigma$-сеть), умножает состояние на этот вектор "забывания"
 * Input Gate - формирует новую информацию на основе $\tanh$-сети и выбирает с помощью второй $\sigma$-сети, что необходимо вставить в состояние
 * Output Gate - на основе состояния с помощью второй $\tanh$-сети формирует выходной вектор, и с помощью третьей $\sigma$-сети определяет, какую его часть выдать на выход.

## Попробуем LSTM в нашем примере

In [18]:
input_var = sequence.input_variable(vocab_size)
label_var = C.input_variable(2)

model = Sequential([Recurrence(LSTM(200)),sequence.last,Dense(100,activation=C.relu),Dense(2)])

z = model(input_var)
z_sm = C.softmax(z)

In [19]:
ce = cross_entropy_with_softmax(z, label_var)
errs = classification_error(z, label_var)

lr_per_sample = learning_rate_schedule(0.02, UnitType.minibatch)
learner = C.learners.adam(z.parameters, lr_per_sample,momentum=C.momentum_as_time_constant_schedule(0.9))
progress_printer = ProgressPrinter(freq=100, tag='Training')
trainer = Trainer(z, (ce, errs), learner, progress_printer)

log_number_of_parameters(z)

minibatch_size = 10

Training 201902 parameters in 7 parameter tensors.


In [20]:
for ep in range(20):
    print("Epoch={}".format(ep))
    for mb in range(0, len(words), minibatch_size):
        trainer.train_minibatch(
            {input_var: words_arr1[mb:mb + minibatch_size], label_var: labels_arr[mb:mb + minibatch_size]})

Epoch=0
Learning rate per minibatch: 0.02
 Minibatch[   1- 100]: loss = 0.623565 * 1000, metric = 29.30% * 1000;
 Minibatch[ 101- 200]: loss = 0.586816 * 1000, metric = 27.40% * 1000;
 Minibatch[ 201- 300]: loss = 0.632443 * 1000, metric = 32.20% * 1000;
 Minibatch[ 301- 400]: loss = 0.594684 * 1000, metric = 27.90% * 1000;
 Minibatch[ 401- 500]: loss = 0.609258 * 1000, metric = 30.10% * 1000;
 Minibatch[ 501- 600]: loss = 0.624320 * 1000, metric = 29.30% * 1000;
Epoch=1
 Minibatch[ 601- 700]: loss = 0.607080 * 994, metric = 30.08% * 994;
 Minibatch[ 701- 800]: loss = 0.580789 * 1000, metric = 27.60% * 1000;
 Minibatch[ 801- 900]: loss = 0.619337 * 1000, metric = 32.10% * 1000;
 Minibatch[ 901-1000]: loss = 0.587785 * 1000, metric = 27.50% * 1000;
 Minibatch[1001-1100]: loss = 0.613588 * 1000, metric = 31.00% * 1000;
 Minibatch[1101-1200]: loss = 0.598917 * 1000, metric = 29.90% * 1000;
Epoch=2
 Minibatch[1201-1300]: loss = 0.590163 * 994, metric = 28.87% * 994;
 Minibatch[1301-1400]: 

 Minibatch[11401-11500]: loss = 0.089188 * 1000, metric = 3.10% * 1000;
Epoch=19
 Minibatch[11501-11600]: loss = 0.119828 * 994, metric = 3.72% * 994;
 Minibatch[11601-11700]: loss = 0.115898 * 1000, metric = 3.90% * 1000;
 Minibatch[11701-11800]: loss = 0.125766 * 1000, metric = 4.10% * 1000;
 Minibatch[11801-11900]: loss = 0.130914 * 1000, metric = 4.40% * 1000;
 Minibatch[11901-12000]: loss = 0.088680 * 1000, metric = 3.60% * 1000;
 Minibatch[12001-12100]: loss = 0.077637 * 1000, metric = 2.70% * 1000;


## Точность с использованием LSTM

In [22]:
check(z_sm,False)

600 out of 725 correct (82.75862068965517%)



## Полезно почитать

  * [Andrey Karpathy. Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
  * [Christopher Olah. Understanding LSTM Networks.](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)

## Выводы

  * RNN - это архитектура сети, предназначенная для анализа/обработки последовательностей переменной длины
  * Идея - формирование по последовательности вектора состояния
  * Не только обработка, но и генерация
  * Очень часто используются LSTM-ячейки
  * CNTK лучше всех фреймворков для работы с RNN за счет динамических размерностей